# Exploring Ebay Car Sales Data

The project is aimed to clean and analyze the dataset of used cars from eBay Kleinanzeigen (German eBay website). The dataset includes 50,000 data points from the original dataset uploaded to Kaggle and has such information as name of the car, the barnd and type of the car, price on the ad, date it was listed, postal code of the location and etc.

In [81]:
import pandas as pd
autos = pd.read_csv("autos.csv", encoding= "Latin-1")

In [82]:
autos

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50
5,2016-03-21 13:47:45,Chrysler_Grand_Voyager_2.8_CRD_Aut.Limited_Sto...,privat,Angebot,"$7,900",test,bus,2006,automatik,150,voyager,"150,000km",4,diesel,chrysler,NaN,2016-03-21 00:00:00,0,22962,2016-04-06 09:45:21
6,2016-03-20 17:55:21,VW_Golf_III_GT_Special_Electronic_Green_Metall...,privat,Angebot,$300,test,limousine,1995,manuell,90,golf,"150,000km",8,benzin,volkswagen,NaN,2016-03-20 00:00:00,0,31535,2016-03-23 02:48:59
7,2016-03-16 18:55:19,Golf_IV_1.9_TDI_90PS,privat,Angebot,"$1,990",control,limousine,1998,manuell,90,golf,"150,000km",12,diesel,volkswagen,nein,2016-03-16 00:00:00,0,53474,2016-04-07 03:17:32
8,2016-03-22 16:51:34,Seat_Arosa,privat,Angebot,$250,test,NaN,2000,manuell,0,arosa,"150,000km",10,NaN,seat,nein,2016-03-22 00:00:00,0,7426,2016-03-26 18:18:10
9,2016-03-16 13:47:02,Renault_Megane_Scenic_1.6e_RT_Klimaanlage,privat,Angebot,$590,control,bus,1997,manuell,90,megane,"150,000km",7,benzin,renault,nein,2016-03-16 00:00:00,0,15749,2016-04-06 10:46:35


In [83]:
print(autos.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
dateCrawled            50000 non-null object
name                   50000 non-null object
seller                 50000 non-null object
offerType              50000 non-null object
price                  50000 non-null object
abtest                 50000 non-null object
vehicleType            44905 non-null object
yearOfRegistration     50000 non-null int64
gearbox                47320 non-null object
powerPS                50000 non-null int64
model                  47242 non-null object
odometer               50000 non-null object
monthOfRegistration    50000 non-null int64
fuelType               45518 non-null object
brand                  50000 non-null object
notRepairedDamage      40171 non-null object
dateCreated            50000 non-null object
nrOfPictures           50000 non-null int64
postalCode             50000 non-null int64
lastSeen               50000 non-null obj

In [84]:
print(autos.head())

           dateCrawled                                               name  \
0  2016-03-26 17:47:46                   Peugeot_807_160_NAVTECH_ON_BOARD   
1  2016-04-04 13:38:56         BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik   
2  2016-03-26 18:57:24                         Volkswagen_Golf_1.6_United   
3  2016-03-12 16:58:10  Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...   
4  2016-04-01 14:38:50  Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...   

   seller offerType   price   abtest vehicleType  yearOfRegistration  \
0  privat   Angebot  $5,000  control         bus                2004   
1  privat   Angebot  $8,500  control   limousine                1997   
2  privat   Angebot  $8,990     test   limousine                2009   
3  privat   Angebot  $4,350  control  kleinwagen                2007   
4  privat   Angebot  $1,350     test       kombi                2003   

     gearbox  powerPS   model   odometer  monthOfRegistration fuelType  \
0    manuell      158  andere 

Based on the info() method we could observe that the dataset. contains 20 columns (most of them are strings. There are certain columns that have less than 50,000 lines, which means they have null values (but none have more that ~20% null values) and they should be observed in more details. These columns include vehicle type, gearbox, fuel type, model and the column stating if the car has the damage which is not yet repaired.
Moreover, there is a column "price" that has the data type as object, instead of the integer or float. Therefore, it has to be modified going forward in the cleaning process.
Based on the observation of the first 5 rows of the dataset, we can see that certain columns are in Gertman and we might need to translate some of them to English. The column with the number of pictures should also be observed in more details as it seems that first 5 rows did not have any photos (and if all values are zero, to delete that column as it brings to value to the analysis).
Moreover, the column names use the camelcase instead of Python's preferred snakecase. 

In [85]:
autos.columns

Index(['dateCrawled', 'name', 'seller', 'offerType', 'price', 'abtest',
       'vehicleType', 'yearOfRegistration', 'gearbox', 'powerPS', 'model',
       'odometer', 'monthOfRegistration', 'fuelType', 'brand',
       'notRepairedDamage', 'dateCreated', 'nrOfPictures', 'postalCode',
       'lastSeen'],
      dtype='object')

In [86]:
autos.columns = ['date_crawled', 
          'name', 
          'seller', 
          'offer_type', 
          'price', 
          'abtest',
          'vehicle_type', 
          'registration_year',
          'gearbox',
          'power_ps',
          'model',
          'odometer',
          'registration_month',
          'fuel_type',
          'brand',
          'unrepaired_damage',
          'ad_created',
          'num_of_pictures',
          'postal_code',
          'last_seen']


autos.head()

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,num_of_pictures,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


Due to the fact that converting column names from camelcase to snakecase cannot be made by replacing spaces with underscores, we renamed each column name individually and reword some of them based on the data disctionary to make it more decriptive. 

In [87]:
autos.describe(include ='all')

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,num_of_pictures,postal_code,last_seen
count,50000,50000,50000,50000,50000,50000,44905,50000.000000,47320,50000.000000,47242,50000,50000.000000,45518,50000,40171,50000,50000.0,50000.000000,50000
unique,48213,38754,2,2,2357,2,8,NaN,2,NaN,245,13,NaN,7,40,2,76,NaN,NaN,39481
top,2016-03-21 16:37:21,Ford_Fiesta,privat,Angebot,$0,test,limousine,NaN,manuell,NaN,golf,"150,000km",NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-07 06:17:27
freq,3,78,49999,49999,1421,25756,12859,NaN,36993,NaN,4024,32424,NaN,30107,10687,35232,1946,NaN,NaN,8
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005.073280,NaN,116.355920,NaN,NaN,5.723360,NaN,NaN,NaN,NaN,0.0,50813.627300,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.712813,NaN,209.216627,NaN,NaN,3.711984,NaN,NaN,NaN,NaN,0.0,25779.747957,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999.000000,NaN,70.000000,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,0.0,30451.000000,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003.000000,NaN,105.000000,NaN,NaN,6.000000,NaN,NaN,NaN,NaN,0.0,49577.000000,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.000000,NaN,150.000000,NaN,NaN,9.000000,NaN,NaN,NaN,NaN,0.0,71540.000000,NaN


We are using the DataFrame.describe() method to help us explore the dataset. We can observe that the column 'num_of_pictures' might have mostly one value as zero and is a candidate to be dropped. Columns that have less than 50,000 rows might need more investigation to figure out the reason for the null values. 'Seller' and 'offer_type' columns have almost all the same values. Moreover, numeric data stored as text in columns 'price' and 'odometer' need to be cleaned.

In [88]:
autos["num_of_pictures"].value_counts()

0    50000
Name: num_of_pictures, dtype: int64

We used the Series.value_counts() method to have a closer look at the "num_of_pictures" column. As was assumed before, the whole column has zero values. Therefore, it should be dropped as it does not have useful information for the analysis.


In [89]:
autos = autos.drop("num_of_pictures", axis=1)

In [90]:
autos

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,39218,2016-04-01 14:38:50
5,2016-03-21 13:47:45,Chrysler_Grand_Voyager_2.8_CRD_Aut.Limited_Sto...,privat,Angebot,"$7,900",test,bus,2006,automatik,150,voyager,"150,000km",4,diesel,chrysler,NaN,2016-03-21 00:00:00,22962,2016-04-06 09:45:21
6,2016-03-20 17:55:21,VW_Golf_III_GT_Special_Electronic_Green_Metall...,privat,Angebot,$300,test,limousine,1995,manuell,90,golf,"150,000km",8,benzin,volkswagen,NaN,2016-03-20 00:00:00,31535,2016-03-23 02:48:59
7,2016-03-16 18:55:19,Golf_IV_1.9_TDI_90PS,privat,Angebot,"$1,990",control,limousine,1998,manuell,90,golf,"150,000km",12,diesel,volkswagen,nein,2016-03-16 00:00:00,53474,2016-04-07 03:17:32
8,2016-03-22 16:51:34,Seat_Arosa,privat,Angebot,$250,test,NaN,2000,manuell,0,arosa,"150,000km",10,NaN,seat,nein,2016-03-22 00:00:00,7426,2016-03-26 18:18:10
9,2016-03-16 13:47:02,Renault_Megane_Scenic_1.6e_RT_Klimaanlage,privat,Angebot,$590,control,bus,1997,manuell,90,megane,"150,000km",7,benzin,renault,nein,2016-03-16 00:00:00,15749,2016-04-06 10:46:35


Let's now convert 'price' and 'odometer' columns form text to numeric values. In order to do this we will remove any non-numeric characters and convert the column to a numeric dtype.

In [91]:
autos["price"].head()

0    $5,000
1    $8,500
2    $8,990
3    $4,350
4    $1,350
Name: price, dtype: object

In [92]:
autos["price"] = (autos["price"]
                  .str.replace('$','')
                  .str.replace(',','')
                  .astype(int))

In [93]:
autos["odometer"].head()

0    150,000km
1    150,000km
2     70,000km
3     70,000km
4    150,000km
Name: odometer, dtype: object

In [94]:
autos["odometer"] = (autos["odometer"]
                         .str.replace('km','')
                         .str.replace(',','')
                         .astype(int))

autos.rename({"odometer":"odometer_km"}, axis=1,inplace=True)
print(autos["odometer_km"].head())

0    150000
1    150000
2     70000
3     70000
4    150000
Name: odometer_km, dtype: int64


In [95]:
#how many unique values
print(autos["price"].unique().shape)
print(autos["odometer_km"].unique().shape)

(2357,)
(13,)


In [96]:
print(autos["price"].describe())
print(autos["price"].value_counts().head())
print(autos["price"].value_counts().sort_index(ascending=False).head(10))
print(autos["price"].value_counts().sort_index(ascending=True).head(10))

count    5.000000e+04
mean     9.840044e+03
std      4.811044e+05
min      0.000000e+00
25%      1.100000e+03
50%      2.950000e+03
75%      7.200000e+03
max      1.000000e+08
Name: price, dtype: float64
0       1421
500      781
1500     734
2500     643
1000     639
Name: price, dtype: int64
99999999    1
27322222    1
12345678    3
11111111    2
10000000    1
3890000     1
1300000     1
1234566     1
999999      2
999990      1
Name: price, dtype: int64
0     1421
1      156
2        3
3        1
5        2
8        1
9        1
10       7
11       2
12       3
Name: price, dtype: int64


In [97]:
print(autos["odometer_km"].describe())
print(autos["odometer_km"].value_counts(dropna=False).head())
print(autos["odometer_km"].value_counts(dropna=False).sort_index(ascending=False).head())
print(autos["odometer_km"].value_counts().sort_index(ascending=True).head())

count     50000.000000
mean     125732.700000
std       40042.211706
min        5000.000000
25%      125000.000000
50%      150000.000000
75%      150000.000000
max      150000.000000
Name: odometer_km, dtype: float64
150000    32424
125000     5170
100000     2169
90000      1757
80000      1436
Name: odometer_km, dtype: int64
150000    32424
125000     5170
100000     2169
90000      1757
80000      1436
Name: odometer_km, dtype: int64
5000     967
10000    264
20000    784
30000    789
40000    819
Name: odometer_km, dtype: int64


Based on the column analysis above, we can see that the minimal value for the 'price' column is zero, which is unrealistic. Moreover, there are 1421 rows that have zero value, therefore, they have to be researched a little bit more. There are also some values with the price being unrealistically high that need to be looked in more details.

In [98]:
print(autos.loc[autos["price"] == 0,"price"])
print(autos.loc[autos["price"] == 99999999, "price"])
print(autos.loc[autos["price"] == 1,"price"])
print(autos.loc[autos["price"] == 27322222,"price"])

27       0
71       0
80       0
87       0
99       0
118      0
146      0
167      0
180      0
226      0
234      0
248      0
259      0
301      0
323      0
327      0
347      0
366      0
389      0
418      0
427      0
430      0
449      0
487      0
502      0
519      0
565      0
593      0
660      0
661      0
        ..
48882    0
48889    0
48891    0
48982    0
48985    0
49043    0
49082    0
49254    0
49301    0
49342    0
49350    0
49404    0
49408    0
49479    0
49496    0
49504    0
49507    0
49525    0
49538    0
49551    0
49614    0
49739    0
49755    0
49793    0
49880    0
49884    0
49943    0
49960    0
49974    0
49984    0
Name: price, Length: 1421, dtype: int64
39705    99999999
Name: price, dtype: int64
55       1
344      1
791      1
981      1
1115     1
1422     1
1561     1
1877     1
1987     1
2442     1
2865     1
2892     1
3062     1
3397     1
3611     1
3679     1
4194     1
4253     1
4323     1
4496     1
4632     1
4787     1
492

In [99]:
print(autos.loc[27])
print(autos.loc[39705])
print(autos.loc[42221])
print(autos.loc[6013])

date_crawled                             2016-03-27 18:45:01
name                  Hat_einer_Ahnung_mit_Ford_Galaxy_HILFE
seller                                                privat
offer_type                                           Angebot
price                                                      0
abtest                                               control
vehicle_type                                             NaN
registration_year                                       2005
gearbox                                                  NaN
power_ps                                                   0
model                                                    NaN
odometer_km                                           150000
registration_month                                         0
fuel_type                                                NaN
brand                                                   ford
unrepaired_damage                                        NaN
ad_created              

After having a deeper research on the outliers (0 and 99,999,999) we can see that they were created by mistake. Therefore, they are to be removed. Some of other prices also could be seen as quite low or too high (like $1 or $10,000,000). However, there is a possibility that $1 cars were just to be given away on the website. While the more expensive price is associated with the luxury car model and the rare year.

In [100]:
autos[autos["price"].between(0,99999999, inclusive=False )]

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,5000,control,bus,2004,manuell,158,andere,150000,3,lpg,peugeot,nein,2016-03-26 00:00:00,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,8500,control,limousine,1997,automatik,286,7er,150000,6,benzin,bmw,nein,2016-04-04 00:00:00,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,8990,test,limousine,2009,manuell,102,golf,70000,7,benzin,volkswagen,nein,2016-03-26 00:00:00,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,4350,control,kleinwagen,2007,automatik,71,fortwo,70000,6,benzin,smart,nein,2016-03-12 00:00:00,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,1350,test,kombi,2003,manuell,0,focus,150000,7,benzin,ford,nein,2016-04-01 00:00:00,39218,2016-04-01 14:38:50
5,2016-03-21 13:47:45,Chrysler_Grand_Voyager_2.8_CRD_Aut.Limited_Sto...,privat,Angebot,7900,test,bus,2006,automatik,150,voyager,150000,4,diesel,chrysler,NaN,2016-03-21 00:00:00,22962,2016-04-06 09:45:21
6,2016-03-20 17:55:21,VW_Golf_III_GT_Special_Electronic_Green_Metall...,privat,Angebot,300,test,limousine,1995,manuell,90,golf,150000,8,benzin,volkswagen,NaN,2016-03-20 00:00:00,31535,2016-03-23 02:48:59
7,2016-03-16 18:55:19,Golf_IV_1.9_TDI_90PS,privat,Angebot,1990,control,limousine,1998,manuell,90,golf,150000,12,diesel,volkswagen,nein,2016-03-16 00:00:00,53474,2016-04-07 03:17:32
8,2016-03-22 16:51:34,Seat_Arosa,privat,Angebot,250,test,NaN,2000,manuell,0,arosa,150000,10,NaN,seat,nein,2016-03-22 00:00:00,7426,2016-03-26 18:18:10
9,2016-03-16 13:47:02,Renault_Megane_Scenic_1.6e_RT_Klimaanlage,privat,Angebot,590,control,bus,1997,manuell,90,megane,150000,7,benzin,renault,nein,2016-03-16 00:00:00,15749,2016-04-06 10:46:35


After selecting the range between zero and 99,999,999 (not including), we have 48,578 rows left.

# Exploring the date columns

In [101]:
#Extracting just the date values
autos["date_crawled"] = autos["date_crawled"].str.split().str[0]
autos["ad_created"] = autos["ad_created"].str.split().str[0]
autos["last_seen"] = autos["last_seen"].str.split().str[0]

print(autos["date_crawled"].value_counts(normalize=True, dropna=False)
      .sort_index(ascending=True)[:10],"\n")

print(autos["ad_created"].value_counts(normalize=True, dropna=False)
      .sort_index(ascending=True)[:10])
print()
print(autos["last_seen"].value_counts(normalize=True, dropna=False)
      .sort_index(ascending=True)[:10])

2016-03-05    0.02538
2016-03-06    0.01394
2016-03-07    0.03596
2016-03-08    0.03330
2016-03-09    0.03322
2016-03-10    0.03212
2016-03-11    0.03248
2016-03-12    0.03678
2016-03-13    0.01556
2016-03-14    0.03662
Name: date_crawled, dtype: float64 

2015-06-11    0.00002
2015-08-10    0.00002
2015-09-09    0.00002
2015-11-10    0.00002
2015-12-05    0.00002
2015-12-30    0.00002
2016-01-03    0.00002
2016-01-07    0.00002
2016-01-10    0.00004
2016-01-13    0.00002
Name: ad_created, dtype: float64

2016-03-05    0.00108
2016-03-06    0.00442
2016-03-07    0.00536
2016-03-08    0.00760
2016-03-09    0.00986
2016-03-10    0.01076
2016-03-11    0.01252
2016-03-12    0.02382
2016-03-13    0.00898
2016-03-14    0.01280
Name: last_seen, dtype: float64


After analysing the 'date_crawled' and 'last_seen' columns we can observe that the vast majority was created in March, 2016. This could be expalined by the fast that these columns were added by the crawler.

In [102]:
autos["registration_year"].describe()

count    50000.000000
mean      2005.073280
std        105.712813
min       1000.000000
25%       1999.000000
50%       2003.000000
75%       2008.000000
max       9999.000000
Name: registration_year, dtype: float64

Using the Series.describe() method to analyse the 'registration_year' distribution shows us that the data has some odd values like 1000 or 9999, which are unrealistic years for our dataset. 

In [103]:
autos[autos["registration_year"].between(1000,1900, inclusive=False)]

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,postal_code,last_seen
10556,2016-04-01,UNFAL_Auto,privat,Angebot,450,control,NaN,1800,NaN,1800,NaN,5000,2,NaN,mitsubishi,nein,2016-04-01,63322,2016-04-01
24511,2016-03-17,Trabant__wartburg__Ostalgie,privat,Angebot,490,control,NaN,1111,NaN,0,NaN,5000,0,NaN,trabant,NaN,2016-03-17,16818,2016-04-07
32585,2016-04-02,UNFAL_Auto,privat,Angebot,450,control,NaN,1800,NaN,1800,NaN,5000,2,NaN,mitsubishi,nein,2016-04-02,63322,2016-04-04
35238,2016-03-26,Suche_Skoda_Fabia____Skoda_Fabia_Combi_mit_Klima,privat,Angebot,0,control,NaN,1500,NaN,0,NaN,5000,0,benzin,skoda,NaN,2016-03-26,15517,2016-04-04
49283,2016-03-15,Citroen_HY,privat,Angebot,7750,control,NaN,1001,NaN,0,andere,5000,0,NaN,citroen,NaN,2016-03-15,66706,2016-04-06


In [104]:
autos[autos["registration_year"].between(2016,9999, inclusive=False)]


,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,postal_code,last_seen
10,2016-03-15,VW_Golf_Tuning_in_siber/grau,privat,Angebot,999,test,NaN,2017,manuell,90,NaN,150000,4,benzin,volkswagen,nein,2016-03-14,86157,2016-04-07
55,2016-03-07,Mercedes_E320_AMG_zu_Tauschen!,privat,Angebot,1,test,NaN,2017,automatik,224,e_klasse,125000,7,benzin,mercedes_benz,nein,2016-03-06,22111,2016-03-08
65,2016-04-04,Ford_Fiesta_zum_ausschlachten,privat,Angebot,250,control,NaN,2017,manuell,65,fiesta,125000,9,benzin,ford,NaN,2016-04-04,65606,2016-04-05
68,2016-04-03,Mini_cooper_s_clubman_/vollausstattung_/_Navi/...,privat,Angebot,10990,test,NaN,2017,manuell,174,clubman,100000,0,NaN,mini,nein,2016-04-03,83135,2016-04-05
84,2016-03-27,Renault_twingo,privat,Angebot,900,control,NaN,2018,NaN,60,twingo,150000,0,NaN,renault,NaN,2016-03-27,40589,2016-04-05
113,2016-04-03,Golf_4_Anfaenger_auto,privat,Angebot,1200,test,NaN,2017,manuell,75,golf,150000,7,NaN,volkswagen,NaN,2016-04-03,97656,2016-04-05
164,2016-03-13,Opel_Meriva__nur_76000_Km__unfallfrei__scheckh...,privat,Angebot,4800,control,NaN,2018,manuell,0,meriva,80000,4,benzin,opel,nein,2016-03-13,37627,2016-04-04
197,2016-04-05,VW_Polo_9N_an_Bastler,privat,Angebot,888,control,NaN,2017,manuell,64,polo,20000,7,NaN,volkswagen,ja,2016-04-05,58566,2016-04-07
253,2016-03-27,Ford_mondeo_Gas_anlage_mit_TÜV_04.2017,privat,Angebot,2250,test,NaN,2017,manuell,0,mondeo,150000,8,benzin,ford,nein,2016-03-27,56575,2016-04-05
348,2016-03-17,VW_Beetle_1.8Turbo_mit_Vollausstattung_und_seh...,privat,Angebot,3750,control,NaN,2017,manuell,150,beetle,150000,7,NaN,volkswagen,nein,2016-03-17,45896,2016-03-24


Due to the fact that the car cannot be registered after the listing was seen, we suppose that a registration year above 2016 would be inaccurate. Therefore, we have checked the number of listings that fall outside the 1900-2016 interval to see if it is safe to remove thise rows entirely. Couting the intervals from 1000 - 1900 and 2016 - 9999 gives us 5 and 1962 rows respectively, which altogether is approximately 4% of the total dataset. In order to make our dataset more clean and accurate, removing those rows should not have any significantly influence.
Therefore, we are keeping the values in the interval between 1900 - 2016.

In [105]:
autos = autos[autos["registration_year"].between(1900,2016, inclusive=True)]

In [106]:
autos

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,postal_code,last_seen
0,2016-03-26,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,5000,control,bus,2004,manuell,158,andere,150000,3,lpg,peugeot,nein,2016-03-26,79588,2016-04-06
1,2016-04-04,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,8500,control,limousine,1997,automatik,286,7er,150000,6,benzin,bmw,nein,2016-04-04,71034,2016-04-06
2,2016-03-26,Volkswagen_Golf_1.6_United,privat,Angebot,8990,test,limousine,2009,manuell,102,golf,70000,7,benzin,volkswagen,nein,2016-03-26,35394,2016-04-06
3,2016-03-12,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,4350,control,kleinwagen,2007,automatik,71,fortwo,70000,6,benzin,smart,nein,2016-03-12,33729,2016-03-15
4,2016-04-01,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,1350,test,kombi,2003,manuell,0,focus,150000,7,benzin,ford,nein,2016-04-01,39218,2016-04-01
5,2016-03-21,Chrysler_Grand_Voyager_2.8_CRD_Aut.Limited_Sto...,privat,Angebot,7900,test,bus,2006,automatik,150,voyager,150000,4,diesel,chrysler,NaN,2016-03-21,22962,2016-04-06
6,2016-03-20,VW_Golf_III_GT_Special_Electronic_Green_Metall...,privat,Angebot,300,test,limousine,1995,manuell,90,golf,150000,8,benzin,volkswagen,NaN,2016-03-20,31535,2016-03-23
7,2016-03-16,Golf_IV_1.9_TDI_90PS,privat,Angebot,1990,control,limousine,1998,manuell,90,golf,150000,12,diesel,volkswagen,nein,2016-03-16,53474,2016-04-07
8,2016-03-22,Seat_Arosa,privat,Angebot,250,test,NaN,2000,manuell,0,arosa,150000,10,NaN,seat,nein,2016-03-22,7426,2016-03-26
9,2016-03-16,Renault_Megane_Scenic_1.6e_RT_Klimaanlage,privat,Angebot,590,control,bus,1997,manuell,90,megane,150000,7,benzin,renault,nein,2016-03-16,15749,2016-04-06


In [107]:
print(autos["registration_year"].value_counts(normalize = True, dropna = False))
print(autos["registration_year"].value_counts(normalize = True, dropna = False)
                                .sort_index(ascending=True)[:10],"\n")
print(autos["registration_year"].value_counts(normalize = True, dropna = False)
                                .sort_index(ascending=False)[:10],"\n")




2000    0.069834
2005    0.062776
1999    0.062464
2004    0.056988
2003    0.056779
2006    0.056384
2001    0.056280
2002    0.052740
1998    0.051074
2007    0.047972
2008    0.046452
2009    0.043683
1997    0.042225
2011    0.034022
2010    0.033251
1996    0.030066
2012    0.027546
2016    0.027401
1995    0.027338
2013    0.016782
2014    0.013867
1994    0.013742
1993    0.009265
2015    0.008308
1990    0.008224
1992    0.008141
1991    0.007412
1989    0.003769
1988    0.002957
1985    0.002186
          ...   
1966    0.000458
1977    0.000458
1975    0.000396
1969    0.000396
1965    0.000354
1964    0.000250
1910    0.000187
1963    0.000187
1959    0.000146
1961    0.000125
1956    0.000104
1958    0.000083
1937    0.000083
1962    0.000083
1950    0.000062
1954    0.000042
1941    0.000042
1934    0.000042
1957    0.000042
1951    0.000042
1955    0.000042
1931    0.000021
1953    0.000021
1943    0.000021
1938    0.000021
1939    0.000021
1927    0.000021
1929    0.0000

After analysing the cleaned dataset we can observe that the highest percentage ~7% of the cars were registered in 2000 and the lowest in 1952. The oldest registration year is 1910.

In [108]:
print(autos["brand"].unique(),'\n')
print(autos["brand"].value_counts().head(20))
print(autos["brand"].value_counts().head(20).index)


['peugeot' 'bmw' 'volkswagen' 'smart' 'ford' 'chrysler' 'seat' 'renault'
 'mercedes_benz' 'audi' 'sonstige_autos' 'opel' 'mazda' 'porsche' 'mini'
 'toyota' 'dacia' 'nissan' 'jeep' 'saab' 'volvo' 'mitsubishi' 'jaguar'
 'fiat' 'skoda' 'subaru' 'kia' 'citroen' 'chevrolet' 'hyundai' 'honda'
 'daewoo' 'suzuki' 'trabant' 'land_rover' 'alfa_romeo' 'lada' 'rover'
 'daihatsu' 'lancia'] 

volkswagen        10188
bmw                5284
opel               5195
mercedes_benz      4580
audi               4149
ford               3352
renault            2274
peugeot            1418
fiat               1242
seat                873
skoda               770
mazda               727
nissan              725
citroen             669
smart               668
toyota              599
sonstige_autos      526
hyundai             473
volvo               444
mini                415
Name: brand, dtype: int64
Index(['volkswagen', 'bmw', 'opel', 'mercedes_benz', 'audi', 'ford', 'renault',
       'peugeot', 'fiat', 'seat'

In order to explore variations across different car brands, we are exploring unqiue values in the 'brand' column and chooseing the top 20 brands to aggregate by. The top brand in our dataset is Volkswagen and the least out of 20 is Mini.

In [109]:
selected_brands = autos["brand"].value_counts().head(20).index
brand_dict = {}

for b in selected_brands:
    selected_rows = autos[autos["brand"] == b]
    mean_price = selected_rows["price"].mean()
    brand_dict[b] = mean_price
    
print(brand_dict)



{'toyota': 5115.33388981636, 'peugeot': 3039.4682651622, 'fiat': 2711.8011272141707, 'ford': 7263.015811455847, 'mini': 10460.012048192772, 'mercedes_benz': 30317.447816593885, 'bmw': 8334.645155185466, 'volvo': 4757.108108108108, 'volkswagen': 6516.457597173145, 'sonstige_autos': 39621.77946768061, 'mazda': 4010.7716643741405, 'hyundai': 5308.53911205074, 'seat': 4296.492554410081, 'nissan': 4664.891034482758, 'audi': 9093.65003615329, 'skoda': 6334.91948051948, 'opel': 5252.61655437921, 'smart': 3542.706586826347, 'renault': 2395.4164467897976, 'citroen': 44534.79671150971}


In [110]:
for w in sorted(brand_dict,key = brand_dict.get, reverse =True):
    print (w, brand_dict[w])

citroen 44534.79671150971
sonstige_autos 39621.77946768061
mercedes_benz 30317.447816593885
mini 10460.012048192772
audi 9093.65003615329
bmw 8334.645155185466
ford 7263.015811455847
volkswagen 6516.457597173145
skoda 6334.91948051948
hyundai 5308.53911205074
opel 5252.61655437921
toyota 5115.33388981636
volvo 4757.108108108108
nissan 4664.891034482758
seat 4296.492554410081
mazda 4010.7716643741405
smart 3542.706586826347
peugeot 3039.4682651622
fiat 2711.8011272141707
renault 2395.4164467897976


After aggregating the brand data and sorting it in descending order, we can observe that citroen cars have the highest mean price amount our selected brands, while renault has the lowest mean price.

In [111]:
selected_brands = autos["brand"].value_counts().head(6).index
brand_mean_price = {}
brand_mean_mileage = {}

for b in selected_brands:
    selected_rows = autos[autos["brand"] == b]
    mean_price = selected_rows["price"].mean()
    mean_mileage = selected_rows["odometer_km"].mean()
    brand_mean_price[b] = mean_price
    brand_mean_mileage[b] = mean_mileage
    
print(brand_mean_price,'\n')
print(brand_mean_mileage)

{'audi': 9093.65003615329, 'ford': 7263.015811455847, 'mercedes_benz': 30317.447816593885, 'bmw': 8334.645155185466, 'volkswagen': 6516.457597173145, 'opel': 5252.61655437921} 

{'audi': 129287.78018799711, 'ford': 124046.83770883054, 'mercedes_benz': 130860.26200873363, 'bmw': 132434.70855412565, 'volkswagen': 128730.36906164115, 'opel': 129227.14148219442}


In [112]:
#Converting both dictionaries to series objects, using the series constructor
bmp_series = pd.Series(brand_mean_price)
bmm_series = pd.Series(brand_mean_mileage)

#Creating a dataframe from the first series object using the dataframe constructor

df = pd.DataFrame(bmp_series, columns=['mean_price'])


In [113]:
df

,mean_price
audi,9093.650036
bmw,8334.645155
ford,7263.015811
mercedes_benz,30317.447817
opel,5252.616554
volkswagen,6516.457597


In [114]:
#Assigning the other series as a new column in the dataframe
df['mean_mileage'] = bmm_series

In [115]:
df

,mean_price,mean_mileage
audi,9093.650036,129287.780188
bmw,8334.645155,132434.708554
ford,7263.015811,124046.837709
mercedes_benz,30317.447817,130860.262009
opel,5252.616554,129227.141482
volkswagen,6516.457597,128730.369062


In [118]:
df=df.sort_values("mean_price",ascending=False)
df

,mean_price,mean_mileage
mercedes_benz,30317.447817,130860.262009
audi,9093.650036,129287.780188
bmw,8334.645155,132434.708554
ford,7263.015811,124046.837709
volkswagen,6516.457597,128730.369062
opel,5252.616554,129227.141482


After cretating the dataframe containing both mean price and mean mileage figures, we can see that there is no certain link between price and mileage. Futhermore, average price for mercedes seems to be distinct from other car brands. Looking separately to mercedes figures shows that the dataset also includes many limousines that increase the avearge price.